In [ ]:
import tweepy, codecs
import csv, io
import pandas as pd
import re
from bs4 import BeautifulSoup

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
results = []

In [ ]:
results = api.search(q = "#Brexit -filter:retweets", lang = "en", result_type = "recent", count = 100, tweet_mode='extended')

In [ ]:
quantity = 10000
done = False

minID = results[0].id
for tweet in results[1:len(results)]:
    if tweet.id < minID:
        minID = tweet.id

while not done:
        print("Getting 100 tweets...")

        tempResults = api.search(q = "Brexit -filter:retweets", lang = "en", tweet_mode='extended', count=100, max_id=(minID - 1))

        results = results + tempResults

        if tempResults == [] or len(results) >= quantity:
            done = True

        for tweet in tempResults:
            if tweet.id < minID:
                minID = tweet.id

        print("got", len(tempResults))

In [ ]:
file = codecs.open("Twitter_data_files/brexit.txt", "w", "utf-8")
for result in results:
    file.write(result.full_text)
    file.write("\n")
file.close()

In [ ]:
with io.open('Twitter_data_files/brexit.csv', 'w', encoding='utf8', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["comment_text"])
    with io.open("Twitter_data_files/brexit.txt", 'r', encoding='utf8') as f:
        for tweet in f.readlines():
            tweet = tweet.strip()

            if len(tweet) == 0:
                print('skipped')
                continue
            
            print(tweet)
            csv_writer.writerow([tweet])

# clean

In [ ]:
from nltk.tokenize import WordPunctTokenizer

tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [ ]:
df = pd.read_csv("Twitter_data_files/brexit.csv")

In [ ]:
len(df["comment_text"])

In [ ]:
testing = df.comment_text[0:]
test_result = []

for t in testing:
    test_result.append(tweet_cleaner(t))
test_result

In [ ]:
nums = [0,14000]
print ("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%10000 == 0 ):
        print ("Tweets %d of %d has been processed" % ( i+1, nums[1] ))
    clean_tweet_texts.append(tweet_cleaner(df['comment_text'][i]))

In [ ]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['comment_text'])
clean_df.head()

In [ ]:
clean_df.to_csv('Twitter_data_files/clean_tweet.csv',encoding='utf-8', index=False)
csv = 'Twitter_data_files/clean_tweet.csv'
my_df = pd.read_csv(csv)
my_df.head()

In [ ]:
new_clean_df = my_df.fillna("some value")

In [ ]:
new_clean_df.head()

In [ ]:
new_clean_df.to_csv('Twitter_data_files/new_clean_tweet.csv', encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv('Twitter_data_files/new_clean_tweet.csv')

In [ ]:
df.isnull().sum()